# PLN:

David Ramajo Fernández


**Introducción:** Se realiza el fine-tuning y la comparación del rendimiento de tres modelos Transformer: BERT, RoBERTa y DistilBERT, en la tarea de clasificación de sentimientos utilizando un dataset estándar para esta tarea (yelp_review_full).
La comparación, realizada sobre un conjunto de datos de prueba, permite evaluar las diferencias en rendimiento (BERT vs. RoBERTa) y el impacto de la destilación (BERT vs. DistilBERT) en la calidad de la predicción y potencial eficiencia.





##Se instalan y cargan las librerias necesarias
También se montara google drive para guardar los pesos de los modelos.

In [ ]:
# Conectamos con Google Drive
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
! pip install transformers datasets
! pip install -U accelerate
! pip install -U transformers
! pip install evaluate
! pip install --upgrade datasets fsspec huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import os
import time

os.makedirs('/content/gdrive/MyDrive/seminario14/resultados', exist_ok=True)

## Se carga el conjunto de datos para analisis de sentimientos

Usaremos el conjunto de datos [yelp_review_full](https://huggingface.co/datasets/Yelp/yelp_review_full), donde las reseñas están etiquetadas de 1 a 5 estrellas (etiquetas del 0 al 4).

In [ ]:
dataset = load_dataset("yelp_review_full")

# Se ven 2 ejemplos
print(dataset['train'][0])
print(dataset['train'][1])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'label': 4, 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}
{'label': 1, 'text': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  Y

## Ajuste fino de BERT

### Preparando el conjunto de datos con el tokenizador de BERT

Se usa un 'tokenizador' para preprocesar los datos de texto a un formato que pueda ser introducido en un modelo.

Usaremos el modelo de lenguaje 'bert-base-cased' (y, por lo tanto, su tokenizador usando la clase 'AutoTokenizer').

El método 'map' de 'Datasets' nos permite aplicar una función de preprocesamiento a todo el conjunto de datos.

El conjunto de datos [yelp_review_full](https://huggingface.co/datasets/Yelp/yelp_review_full) es muy grande (700K filas), por lo que usaremos una pequeña porción (20k para entrenamiento, 2k para validación). Se usa una semilla (42) para entrenar y evaluar los 3 modelos con los mismos datos.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(20000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(2000))

# Se ven 2 ejemplos
print(small_train_dataset[0])
print(small_train_dataset[1])

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'label': 4, 'text': "I stalk this truck.  I've been to industrial parks where I pretend to be a tech worker standing in line, strip mall parking lots, and of course the farmer's market.  The bowls are so so absolutely divine.  The owner is super friendly and he makes each bowl by hand with an incredible amount of pride.  You gotta eat here guys!!!", 'input_ids': [101, 146, 27438, 1142, 4202, 119, 146, 112, 1396, 1151, 1106, 3924, 8412, 1187, 146, 9981, 1106, 1129, 170, 13395, 7589, 2288, 1107, 1413, 117, 6322, 8796, 5030, 7424, 117, 1105, 1104, 1736, 1103, 9230, 112, 188, 2319, 119, 1109, 20400, 1132, 1177, 1177, 7284, 10455, 119, 1109, 3172, 1110, 7688, 4931, 1105, 1119, 2228, 1296, 7329, 1118, 1289, 1114, 1126, 10965, 2971, 1104, 8188, 119, 1192, 13224, 3940, 1303, 3713, 106, 106, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Se carga el modelo BERT y el 'head' pre-entrenado del modelo BERT se descarta y se reemplaza por un "head" de clasificación para el dataset yelp_review_full.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Entrenamiento y evaluación de BERT

Se realiza el entrenamiento evaluando la metrica 'accuracy'.

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) # Calcula el índice del valor máximo a lo largo del último eje (que corresponde a la clase predicha) para cada predicción. Esto convierte los logits brutos en predicciones de clase discretas.
    return metric.compute(predictions=predictions, references=labels) # Calcula las métricas de evaluación basándose en las clases predichas (predicciones) y las clases verdaderas (etiquetas).


training_args = TrainingArguments(
    output_dir='/content/gdrive/MyDrive/seminario14/resultados', # Directorio donde se guardarán los resultados
    num_train_epochs=5,              # Número de épocas de entrenamiento
    eval_strategy="epoch",     # Evaluar al final de cada época
    save_strategy="epoch",           # Guardar el modelo al final de cada época
    load_best_model_at_end=True,     # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="accuracy", # Métrica para determinar el mejor modelo
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

start = time.perf_counter()
trainer.train()
print (time.perf_counter() - start)

Epoch,Training Loss,Validation Loss,Accuracy
1,0.973000,0.990135,0.563000
2,0.772100,0.915479,0.611000
3,0.528300,1.058215,0.622000
4,0.347500,1.549631,0.638000
5,0.179300,2.056028,0.636500


9570.333752210001


## Ajuste fino de DistilBERT

### Preparando el conjunto de datos con el tokenizador de DistilBERT

Se realiza igual que el caso anterior, pero para este caso usaremos el modelo de lenguaje 'distilbert-base-cased' (y, por lo tanto, su tokenizador).

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(20000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(2000))

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=5)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Entrenamiento y evaluación de DistilBERT

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) # Calcula el índice del valor máximo a lo largo del último eje (que corresponde a la clase predicha) para cada predicción. Esto convierte los logits brutos en predicciones de clase discretas.
    return metric.compute(predictions=predictions, references=labels) # Calcula las métricas de evaluación basándose en las clases predichas (predicciones) y las clases verdaderas (etiquetas).

training_args = TrainingArguments(
    output_dir='/content/gdrive/MyDrive/seminario14/resultados', # Directorio donde se guardarán los resultados
    num_train_epochs=5,              # Número de épocas de entrenamiento
    eval_strategy="epoch",     # Evaluar al final de cada época
    save_strategy="epoch",           # Guardar el modelo al final de cada época
    load_best_model_at_end=True,     # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="accuracy", # Métrica para determinar el mejor modelo
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

start = time.perf_counter()
trainer.train()
print (time.perf_counter() - start)

Epoch,Training Loss,Validation Loss,Accuracy
1,0.983600,0.948674,0.583500
2,0.788500,0.965160,0.603500
3,0.542800,1.092884,0.610000
4,0.353600,1.589437,0.609500
5,0.218800,2.105110,0.604000


4927.204876293


## Ajuste fino de RoBERTa

### Preparando el conjunto de datos con el tokenizador de RoBERTa

Se realiza igual que en los 2 casos anteriores, pero para este caso usaremos el modelo de lenguaje 'roberta-base' (y, por lo tanto, su tokenizador).

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(20000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(2000))

model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/roberta-base", num_labels=5)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Entrenamiento y evaluación de RoBERTa

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) # Calcula el índice del valor máximo a lo largo del último eje (que corresponde a la clase predicha) para cada predicción. Esto convierte los logits brutos en predicciones de clase discretas.
    return metric.compute(predictions=predictions, references=labels) # Calcula las métricas de evaluación basándose en las clases predichas (predicciones) y las clases verdaderas (etiquetas).

training_args = TrainingArguments(
    output_dir='/content/gdrive/MyDrive/seminario14/resultados', # Directorio donde se guardarán los resultados
    num_train_epochs=5,              # Número de épocas de entrenamiento
    eval_strategy="epoch",     # Evaluar al final de cada época
    save_strategy="epoch",           # Guardar el modelo al final de cada época
    load_best_model_at_end=True,     # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="accuracy", # Métrica para determinar el mejor modelo
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

start = time.perf_counter()
trainer.train()
print (time.perf_counter() - start)

Epoch,Training Loss,Validation Loss,Accuracy
1,0.927600,0.870885,0.625500
2,0.779800,0.982741,0.622000
3,0.598500,0.885722,0.669000
4,0.436800,1.176362,0.645500
5,0.360000,1.664025,0.653500


9812.544007704


## Análisis de Resultados

BERT ofrece una precisión de 0.638, superado únicamente por RoBERTa. Sin embargo, su tiempo por época es de 31.9 minutos, muy similar al de RoBERTa, y tarda 4 épocas en alcanzar su mejor rendimiento. La validación toma 56 segundos, el tiempo más largo de los tres modelos. BERT, al ser el modelo base, sienta las bases para las mejoras implementadas en RoBERTa, pero su mayor número de épocas necesarias y el tiempo de validación sugieren que, si bien es efectivo, puede ser menos eficiente en términos de tiempo total de entrenamiento comparado con DistilBERT.

DistilBERT muestra una alta eficiencia. Con 16.4 minutos por época y una validación en 28 segundos, es significativamente más rápido que BERT y RoBERTa. Esto se debe a su naturaleza "destilada", donde se ha reducido su tamaño y complejidad para ser más rápido y ligero, manteniendo la mayor parte de la capacidad de su modelo original, BERT. Alcanza su mejor resultado en 3 épocas, igual que RoBERTa.
Sin embargo, esta eficiencia tiene un costo en la precisión, ya que DistilBERT obtiene un Accuracy del 0.61. Esta ligera caída en el rendimiento (aproximadamente un 2.8% menos que BERT y un 5.9% menos que RoBERTa) es una compensación esperada por su menor tamaño y mayor velocidad. Para proyectos donde la velocidad de inferencia o los recursos computacionales son limitados, la pequeña reducción en la precisión de DistilBERT podría ser aceptable.

RoBERTa es el modelo con el mejor rendimiento en cuanto a precisión, ha alcanzado un Accuracy del 0.669. Este resultado es ligeramente superior al obtenido por BERT (0.638). Sin embargo, RoBERTa también es el modelo más lento por época, con 32.7 minutos, y su validación toma 53 segundos. Alcanza su mejor resultado en 3 épocas. La mayor precisión de RoBERTa puede atribuirse a su estrategia mejorada del preentrenamiento al abandonar la tarea de predicción de la siguiente oración, que resultó ser poco útil, y en su lugar, se enfoca exclusivamente en el modelado de lenguaje enmascarado. Además, RoBERTa fue entrenado con lotes más grandes, durante un mayor número de épocas y sobre un corpus de texto mucho más extenso que BERT. Estas mejoras permiten a RoBERTa aprender una comprensión más profunda y adaptable del lenguaje, dándole un muy buen resultado en este análisis de sentimientos.

## Conclusiones

Se ha realizado el fine-tuning y la comparación de tres modelos Transformer: BERT, RoBERTa y DistilBERT, para la tarea de clasificación de sentimientos. Los resultados muestran que RoBERTa obtuvo la mayor precisión (0.669), estableciéndose como el modelo de mejor rendimiento, con un tiempo de entrenamiento por época similar al de BERT (aproximadamente 32 minutos). Por su parte, DistilBERT demostró ser el modelo más eficiente, reduciendo significativamente el tiempo de entrenamiento por época (16.4 minutos) y de validación a casi la mitad en comparación con BERT, a expensas de una ligera disminución en la precisión (0.61).

La comparación entre los modelos muestra la importancia de elegir el modelo adecuado según las prioridades necesarias. Mientras que RoBERTa se posiciona como el referente en precisión, su mayor costo computacional debe considerarse. DistilBERT ofrece una alternativa eficiente y ligera, con una pérdida de rendimiento aceptable, lo que lo hace ideal para entornos con recursos limitados.